In [1]:
import tensorflow as tf

from tensorflow.keras.layers import Input, Reshape, Conv2D, BatchNormalization
from tensorflow.keras.layers import MaxPool2D, Dropout, Permute, Flatten, Dense
from tensorflow.keras.models import Model

2025-02-14 20:06:16.999050: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-02-14 20:06:17.004634: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-02-14 20:06:17.019082: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1739559977.048079 1354516 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1739559977.059787 1354516 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-14 20:06:17.092413: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU ins

In [2]:
def load_existing_model() -> Model:
    layers = [1, 2, 3, 4, 5, 6]
    filters = [n * 32 for n in [32, 4, 4, 4, 8, 16]]
    widths = [512, 64, 64, 64, 64, 64]
    strides = [(4, 1), (1, 1), (1, 1), (1, 1), (1, 1), (1, 1)]

    x = Input(shape=(1024,), name='input', dtype='float32')
    y = Reshape(target_shape=(1024, 1, 1), name='input-reshape')(x)

    for l, f, w, s in zip(layers, filters, widths, strides):
        y = Conv2D(f, (w, 1), strides=s, padding='same',
                   activation='relu', name="conv%d" % l)(y)
        y = BatchNormalization(name="conv%d-BN" % l)(y)
        y = MaxPool2D(pool_size=(2, 1), strides=None, padding='valid',
                      name="conv%d-maxpool" % l)(y)
        y = Dropout(0.25, name="conv%d-dropout" % l)(y)

    y = Permute((2, 1, 3), name="transpose")(y)
    y = Flatten(name="flatten")(y)
    y = Dense(360, activation='sigmoid', name="classifier")(y)

    model = Model(inputs=x, outputs=y)

    model.load_weights('crepe-models/model-full.h5')
    model.compile('adam', 'binary_crossentropy')

    return model

In [3]:
full_model = load_existing_model()

2025-02-14 20:06:21.573218: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


In [4]:
full_model.export('crepe-models/full')

INFO:tensorflow:Assets written to: crepe-models/full/assets


INFO:tensorflow:Assets written to: crepe-models/full/assets


Saved artifact at 'crepe-models/full'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 1024), dtype=tf.float32, name='input')
Output Type:
  TensorSpec(shape=(None, 360), dtype=tf.float32, name=None)
Captures:
  124713142250640: TensorSpec(shape=(), dtype=tf.resource, name=None)
  124713142252752: TensorSpec(shape=(), dtype=tf.resource, name=None)
  124713142252176: TensorSpec(shape=(), dtype=tf.resource, name=None)
  124713142253328: TensorSpec(shape=(), dtype=tf.resource, name=None)
  124713142250832: TensorSpec(shape=(), dtype=tf.resource, name=None)
  124713142251408: TensorSpec(shape=(), dtype=tf.resource, name=None)
  124713142253712: TensorSpec(shape=(), dtype=tf.resource, name=None)
  124713142253904: TensorSpec(shape=(), dtype=tf.resource, name=None)
  124713142254288: TensorSpec(shape=(), dtype=tf.resource, name=None)
  124713142253136: TensorSpec(shape=(), dtype=tf.resource, name=None)
  124713142251024: TensorSp

In [5]:
!python -m tf2onnx.convert --saved-model "crepe-models/full" --output "crepe-models/crepe-full.onnx"

2025-02-14 20:06:25.015954: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-02-14 20:06:25.018683: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-02-14 20:06:25.025951: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1739559985.038664 1354618 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1739559985.042531 1354618 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-14 20:06:25.056434: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU ins